In [1]:
###############################
#Started on Thu January 23 2020
#@author: Jonathan Frame
###############################
# This program is designed to generate a synthetic dataset of atmospheric forcing conditions that 
# match the NLDAS data from 1980 - 2014.
# The synthetic record can be extended out as far as the user likes,
# and the user can add nonstationary conditions by adjusting the Parameters controlling the distributions.

In [2]:
%%bash
cat test.conf
echo ""

# This is an example of an input file for MetSim
[MetSim]
#out_vars = ['temp', 'prec', 'shortwave', 'longwave', 'vapor_pressure', 'rel_humid', 'air_pressure', 'wind']

# Time step in minutes
time_step = 180

# Forcings begin here (year/month/day:hour) (hour optional)
start = 1980/1/1

# Forcings end at this date (year/month/day)
stop = 2014/1/31

# Input and output directories
forcing = ./input/cli_test.nc
domain  = ./input/cli_domain.nc
state =   ./input/cli_state.nc

forcing_fmt = netcdf

out_dir = ./output
output_prefix = test

prec_type = triangle

[chunks]
lat = 3
lon = 1

[forcing_vars]
prec  = Prec
t_max = Tmax
t_min = Tmin
wind  = wind

[state_vars]
prec  = prec
t_max = t_max
t_min = t_min

[domain_vars]
lat  = lat
lon  = lon
mask = mask
elev = elev
t_pk = t_pk
dur  = dur



In [5]:
%%bash
source activate metsim_whw

In [7]:
%%bash
ms MetSim-tutorial-master/test.conf -v -n 2

closed dask cluster/client


In [16]:
%pylab inline
import xarray as xr
output = xr.open_dataset('./output/test_19500101-19500131.nc')
output

Populating the interactive namespace from numpy and matplotlib


<xarray.Dataset>
Dimensions:         (lat: 3, lon: 3, time: 248)
Coordinates:
  * time            (time) datetime64[ns] 1950-01-01 ... 1950-01-31T21:00:00
  * lat             (lat) float64 30.03 30.09 30.16
  * lon             (lon) float64 -100.0 -99.97 -99.91
Data variables:
    temp            (time, lat, lon) float32 ...
    prec            (time, lat, lon) float32 ...
    shortwave       (time, lat, lon) float32 ...
    longwave        (time, lat, lon) float32 ...
    vapor_pressure  (time, lat, lon) float32 ...
    rel_humid       (time, lat, lon) float32 ...
Attributes:
    conventions:              1.6
    title:                    Output from MetSim
    institution:              University of Washington
    source:                   metsim.py
    history:                  Created: Fri Jun  5 15:32:35 2020 by jmframe
    references:               Thornton, P.E., and S.W. Running, 1999. An impr...
    comment:                  no comment at this time
    method:                   mtclim
    domain:                   ./input/cli_domain.nc
    state:                    ./input/cli_state.nc
    out_dir:                  /home/NearingLab/projects/jmframe/CAMELS_synthD...
    output_prefix:            test
    start:                    1950-01-01 00:00:00
    stop:                     1950-01-31 00:00:00
    time_step:                180
    calendar:                 standard
    prec_type:                triangle
    out_precision:            f4
    verbose:                  10
    sw_prec_thresh:           0.0
    utc_offset:               False
    lw_cloud:                 cloud_deardorff
    lw_type:                  prata
    tdew_tol:                 1e-06
    tmax_daylength_fraction:  0.67
    rain_scalar:              0.75
    tday_coef:                0.45
    lapse_rate:               0.0065
    out_vars:                 temp, prec, shortwave, longwave, vapor_pressure...
    chunks:                   {lat: 3, lon: 1}
    scheduler:                distributed
    num_workers:              2
    forcing:                  ./input/cli_test.nc
    forcing_fmt:              netcdf
    forcing_vars:             {Prec: prec, Tmax: t_max, Tmin: t_min, wind: wind}
    domain_vars:              {lat: lat, lon: lon, mask: mask, elev: elev, t_...
    state_vars:               {prec: prec, t_max: t_max, t_min: t_min}

In [18]:
import numpy as np
import pandas as pd
import time
import datetime as dt
import sys 
import os
import matplotlib.pyplot as plt
import scipy.stats as ss
from metsim import MetSim
import convert_NLDAS_to_NetCDF

In [9]:
# PROGRAM INITIALIZATION OPTIONS

ymi = 'yearmonth' # Year month index
#  Synthetic data generating parameters
OF = 2 # A multiplier (integer) for continuing out the synthetic forcing data record past the original NLDAS.
PRECIP_PARAMETER1 = 0.75 #  Synthetic data generating parameters
PRECIP_PARAMETER2 = 0.25
TEMP_PARM1 = 0.25
TEMP_PARM2 = 5.8
TEMP_PARM3 = 0.09
TEMP_PARM4 = 5.2
TEMP_PARM5 = 0.13
nGauges = 673 # Setting nGauges to be known number of gauges in CAMELS
count_basins = 0 # just a value to keep track of loops
max_basins = 1 # Stopping the loop after this many basins, for testing the script.
np.set_printoptions(threshold=sys.maxsize)# print options for generating the results files.

In [10]:
#Store all attributes. Really just need basin area, but the rest might come in handy.
att_path = "/home/NearingLab/data/camels_attributes_v2.0/camels_all.txt"
attributes = pd.read_csv(att_path, sep=";")
attributes.set_index('gauge_id', inplace=True)
#data directory path
df_path = "/home/NearingLab/projects/jmframe/CAMELS_synthData/nldas_forcing/"

In [13]:
prefx = '17/'
dataSample = pd.read_csv(df_path+prefx+'13240000_lump_nldas_forcing_leap.txt',
                         sep='\t| ',header=3, engine='python')
dataSample.head()

,Year,Mnth,Day,Hr,Dayl(s),PRCP(mm/day),SRAD(W/m2),SWE(mm),Tmax(C),Tmin(C),Vp(Pa)
0,1980,1,1,12,31104.00,10.83,111.34,0.0,-1.53,-3.23,506.34
1,1980,1,2,12,31104.00,0.63,175.14,0.0,-2.33,-8.18,409.05
2,1980,1,3,12,31206.78,2.93,185.01,0.0,-2.17,-4.09,439.31
3,1980,1,4,12,31446.44,5.14,129.62,0.0,-1.20,-7.01,441.41
4,1980,1,5,12,31449.60,30.66,95.43,0.0,-1.36,-4.88,479.02


In [14]:
count_basins = 0 # just a value to keep track of loops
# Loop through all the files in the forcing data directory.
for filename in os.listdir(df_path):
    print(filename)
    
    nforcing = './input/green_river_forcing.nc'
    ndomain = './input/green_river_domain.nc'
    nstate = './input/green_river_state.nc'
    
    dates = pd.date_range('1980/01/01', '1980/12/31')
    params = {
        'time_step'    : "1440",       
        'start'        : dates[0],
        'stop'         : dates[-1],
        'forcing'      : nforcing,     
        'domain'       : ndomain,
        'state'        : nstate,
        'forcing_fmt'  : 'netcdf',
        'out_dir'      : './output',
        'output_prefix': filename,
        'scheduler'    : 'threading',
        'chunks'       : {'hru': 3},
        'forcing_vars' : {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
        'state_vars'   : {'prcp' : 'prec', 'tmax': 't_max', 'tmin': 't_min'},
        'domain_vars'  : {'elevation': 'elev', 'latitude': 'lat', 'longitude': 'lon', 'mask': 'mask'}
        }
    

    count_basins+=1
    if count_basins >= max_basins:
        break

08


In [15]:
# This is an example of an input file for MetSim
[MetSim]

# Time step in minutes
time_step = 60

# Forcings begin here (year/month/day:hour)
start = 1950/1/1

# Forcings end at this date (year/month/day)
stop = 1950/1/31

# Input specification
forcing = ./MetSim-tutorial-master/input/cli_test.nc
domain  = ./MetSim-tutorial-master/input/cli_domain.nc
state = ./MetSim-tutorial-master/input/cli_state.nc
in_fmt = netcdf

# Output specification
out_dir = ./results
out_prefix = forcing
out_precision = f8

[chunks]
lat = 3
lon = 3

[forcing_vars]
prec = Prec
t_max = Tmax
t_min = Tmin

[state_vars]
prec = prec
t_max = t_max
t_min = t_min

[domain_vars]
lat = lat
lon = lon
mask = mask
elev = elev

SyntaxError: invalid syntax (<ipython-input-15-40b2f3a91a61>, line 14)